In [1]:
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
import xlrd
import time
from IPython.display import Audio

In [2]:
#full runtime indicator start
full_time=time.time()

In [3]:
#finish indicator
wow = '../../Data/Wow.mp3'
finished=Audio(url = wow, autoplay = True) 

In [4]:
#notebook complete indicator
notebook = '../../Data/Hugo Good Job.mp3'
complete=Audio(url = notebook, autoplay = True)

In [5]:
map = gpd.read_file('../../Data/Taxi Zones Shape/taxi_zones.shp').to_crs('epsg:4326')
map = map.drop(['Shape_Area', 'Shape_Leng', 'OBJECTID'], axis=1)


complete

In [6]:
df=pd.read_csv('../../Data/Taxi Data/yellow_14/yellow_tripdata_2014-05.csv')

df.info()

df2=df.head(100)

df2.columns

In [7]:
start_time = time.time()

df['dropoff_geometry'] = df.apply(lambda x: Point((float(x[' dropoff_longitude']), 
                                             float(x[' dropoff_latitude']))), 
                                             axis=1)

print("--- %s seconds ---" % (time.time() - start_time))
finished

--- 906.9426126480103 seconds ---


In [8]:
start_time = time.time()

df['pickup_geometry'] = df.apply(lambda x: Point((float(x[' pickup_longitude']), 
                                             float(x[' pickup_latitude']))), 
                                             axis=1)

print("--- %s seconds ---" % (time.time() - start_time))
finished

--- 903.0416295528412 seconds ---


In [9]:
df.head(3)

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount,dropoff_geometry,pickup_geometry
0,VTS,2014-05-31 17:17:00,2014-05-31 17:17:00,2,0.00,0.00000,0.00000,1,NaN,0.00000,0.00000,CSH,2.5,0.0,0.5,0.00,0.00,3.0,POINT (0 0),POINT (0 0)
1,VTS,2014-05-31 17:07:00,2014-05-31 17:19:00,2,2.22,-73.98704,40.76012,1,NaN,-74.00555,40.74561,CSH,10.5,0.0,0.5,0.00,0.00,11.0,POINT (-74.00555 40.74561),POINT (-73.98703999999998 40.76012)
2,VTS,2014-05-31 16:29:00,2014-05-31 17:17:00,1,17.11,0.00000,0.00000,2,NaN,-73.98220,40.75092,CRD,52.0,0.0,0.5,11.47,5.33,69.3,POINT (-73.98220000000001 40.75092),POINT (0 0)


In [10]:
df2_start=df[[' passenger_count',' trip_distance',' fare_amount',' mta_tax',' tip_amount',' tolls_amount','pickup_geometry']]

In [11]:
df2_end=df[['dropoff_geometry']]

In [12]:
start_time = time.time()
df2_start=gpd.GeoDataFrame(df2_start,
                     crs=map.crs,
                     geometry=df2_start['pickup_geometry'])
print("--- %s seconds ---" % (time.time() - start_time))
finished

--- 120.1375572681427 seconds ---


In [13]:
start_time = time.time()
df2_end=gpd.GeoDataFrame(df2_end,
                     crs=map.crs,
                     geometry=df2_end['dropoff_geometry'])
print("--- %s seconds ---" % (time.time() - start_time))
finished

--- 122.915931224823 seconds ---


In [14]:
start_time = time.time()

geo_processing_start = gpd.sjoin(df2_start, map, op = 'within')

print("--- %s seconds ---" % (time.time() - start_time))
finished

--- 1207.5106754302979 seconds ---


In [15]:
start_time = time.time()

geo_processing_end = gpd.sjoin(df2_end, map, op = 'within')

print("--- %s seconds ---" % (time.time() - start_time))
finished

--- 1262.883705854416 seconds ---


In [16]:
geo_processing_start

,passenger_count,trip_distance,fare_amount,mta_tax,tip_amount,tolls_amount,pickup_geometry,geometry,index_right,zone,LocationID,borough
1,2,2.22,10.5,0.5,0.0,0.0,POINT (-73.98703999999998 40.76012),POINT (-73.98704 40.76012),229,Times Sq/Theatre District,230,Manhattan
8,1,2.96,11.5,0.5,2.4,0.0,POINT (-73.98353 40.75585),POINT (-73.98353 40.75585),229,Times Sq/Theatre District,230,Manhattan
95,3,5.18,18.0,0.5,3.7,0.0,POINT (-73.98679799999998 40.761225),POINT (-73.98680 40.76123),229,Times Sq/Theatre District,230,Manhattan
102,2,0.61,5.0,0.5,1.5,0.0,POINT (-73.98569500000001 40.758247),POINT (-73.98570 40.75825),229,Times Sq/Theatre District,230,Manhattan
117,1,0.00,52.0,0.5,0.0,0.0,POINT (-73.98532799999998 40.760955),POINT (-73.98533 40.76096),229,Times Sq/Theatre District,230,Manhattan
...,...,...,...,...,...,...,...,...,...,...,...,...
11711959,2,5.70,23.5,0.5,0.0,0.0,POINT (-73.82150199999998 40.625292),POINT (-73.82150 40.62529),1,Jamaica Bay,2,Queens
11785430,1,11.20,36.5,0.5,0.0,0.0,POINT (-73.80446999999998 40.617166),POINT (-73.80447 40.61717),1,Jamaica Bay,2,Queens
13902541,1,6.02,25.5,0.5,5.2,0.0,POINT (-73.848212 40.626177),POINT (-73.84821 40.62618),1,Jamaica Bay,2,Queens
10444729,1,3.60,11.5,0.5,0.0,0.0,POINT (-74.12949399999998 40.639633),POINT (-74.12949 40.63963),186,Port Richmond,187,Staten Island


In [17]:
group_start = geo_processing_start.groupby(['LocationID']).describe()

In [18]:
group_start.columns = ['_'.join(col) for col in group_start.columns.values]

group_start.to_csv('../../Data/yellow_tripdata_2014-2-start-processed.csv')

In [19]:
group_end = geo_processing_end.value_counts(['LocationID'])

In [20]:
group_end = pd.DataFrame(group_end,columns=['dropoff_count'])

group_end.to_csv('../../Data/yellow_tripdata_2014-2-end-processed.csv')

In [21]:
merge=group_start.merge(group_end, how='left', on='LocationID')

In [22]:
#REMEMBER TO RENAME CSV BEFORE START
merge.to_csv('../../Data/yellow_tripdata_2014-05-processed.csv')
complete

In [23]:
print("--- %s seconds ---" % (time.time() - full_time))

--- 4590.189940690994 seconds ---
